In [1]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
import pytorch3d.io
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex

import trimesh

from src.util import make_faces

In [2]:
n = 2
start = -0.5
end = 0.5

d1, d2 = torch.meshgrid(
    torch.linspace(start, end, steps=n), 
    torch.linspace(start, end, steps=n))
d3 = torch.full_like(d1, end) + 1 / n
sides =  OrderedDict({
    'front': torch.stack((+d3,  d1,  d2), dim=-1),
    'right': torch.stack(( d1, +d3,  d2), dim=-1),    
    'back' : torch.stack((-d3,  d1,  d2), dim=-1),         
    'left' : torch.stack(( d1, -d3,  d2), dim=-1),
    'top'  : torch.stack(( d1,  d2, +d3), dim=-1),
    'down' : torch.stack(( d1,  d2, -d3), dim=-1),
})
vert = torch.stack(list(sides.values())).reshape(-1, 3)
offset, faces = n ** 2, make_faces(n, n)
faces = torch.cat([
    i * offset + torch.tensor(faces)
    for i in range(6)])
textures = TexturesVertex(verts_features=[torch.ones_like(vert)])
mesh = Meshes(verts=[vert], faces=[faces], textures=textures)
front_f =  [ 0,  1,  2,  3]
right_f =  [ 4,  5,  6,  7]
back_f  =  [ 8,  9, 10, 11]
left_f  =  [12, 13, 14, 15]
top_f   =  [16, 17, 18, 19]
down_f  =  [20, 21, 22, 23]


corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],  
])

used = corners.flatten().tolist()
for f in [17]:#[e for e in front_f if e not in [1]]: # [0]:#    
    for t in [e for e in back_f if e not in used]:
        for l in [e for e in right_f if e not in used]:
            print(f, t, l)
            m = trimesh.Trimesh(vertices=vert, 
                faces=torch.cat((faces, 
                    torch.tensor([
                        [f,  t, l],
                    ]))))
            m.export(f'./data/cube-{f}-{t}-{l}.stl');

In [3]:
corners = torch.tensor([
    [ 0, 22, 14],
    [ 3, 19,  7],
    
    [16,  9, 13],   
    [17, 11,  5],
    
    [ 1, 18, 15],
    [ 2, 23,  6],
    
    [20,  8, 12],
    [21, 10,  4],
])


for (f, t, l) in corners.tolist():
    print(f, t, l)
    m = trimesh.Trimesh(vertices=vert, 
                faces=torch.cat((faces, 
                                 corners)))
    m.export(f'./data/cube-{f}-{t}-{l}.stl');

0 22 14
3 19 7
16 9 13
17 11 5
1 18 15
2 23 6
20 8 12
21 10 4


In [13]:
edges = torch.tensor([
    [ 0, 22,  2],
    [ 2, 22, 23],
    
    [ 3,  7,  2],
    [ 7,  2,  6],
    
    [ 1, 18, 19],
    [ 1, 19,  3],
    
    [ 1, 14, 15],
    [ 1, 14,  0],
    #
    [13, 15, 16],
    [15, 16, 18],
    
    [12, 14, 20],
    [14, 22, 20],
    
    [23, 21,  6],
    [ 6,  4, 21],
    
    [ 7, 19, 17],
    [ 5,  7, 17],
    #
    [11, 16, 17],
    [11,  9, 16],
    
    [8,  9, 13],
    [13, 8, 12],
    
    [10, 4,  5],
    [11, 5, 10],
    
    [10,  8, 21],
    [20, 21,  8],
])


m = trimesh.Trimesh(vertices=vert, 
            faces=torch.cat((faces, 
                             corners,
                             edges,
                            )))
m.export('./data/cube-edge.stl');